In [19]:
import pandas as pd
from sqlalchemy import create_engine


In [30]:
import psycopg2
from io import StringIO

conn = psycopg2.connect(
    dbname="etl_db",
    user="postgres",
    password="postgres",
    host="postgres",
    port=5432,
)
cur = conn.cursor()

total_rows = 0

for i, chunk in enumerate(chunk_iter, start=1):

    # normalize columns
    chunk.columns = (
        chunk.columns
        .str.strip()
        .str.lower()
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
        .str.replace("%", "pct", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace(" ", "_", regex=False)
    )

    buffer = StringIO()
    chunk.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cur.copy_expert(
        f"""
        COPY bronze.us_accidents
        FROM STDIN WITH CSV
        """,
        buffer
    )

    conn.commit()

    rows = len(chunk)
    total_rows += rows
    print(f"Chunk {i}: inserted {rows:,} rows")

cur.close()
conn.close()

print(f"\nIngest complete — {total_rows:,} rows inserted")


Chunk 1: inserted 10,000 rows
Chunk 2: inserted 10,000 rows
Chunk 3: inserted 10,000 rows
Chunk 4: inserted 10,000 rows
Chunk 5: inserted 10,000 rows
Chunk 6: inserted 10,000 rows
Chunk 7: inserted 10,000 rows
Chunk 8: inserted 10,000 rows
Chunk 9: inserted 10,000 rows
Chunk 10: inserted 10,000 rows
Chunk 11: inserted 10,000 rows
Chunk 12: inserted 10,000 rows
Chunk 13: inserted 10,000 rows
Chunk 14: inserted 10,000 rows
Chunk 15: inserted 10,000 rows
Chunk 16: inserted 10,000 rows
Chunk 17: inserted 10,000 rows
Chunk 18: inserted 10,000 rows
Chunk 19: inserted 10,000 rows
Chunk 20: inserted 10,000 rows
Chunk 21: inserted 10,000 rows
Chunk 22: inserted 10,000 rows
Chunk 23: inserted 10,000 rows
Chunk 24: inserted 10,000 rows
Chunk 25: inserted 10,000 rows
Chunk 26: inserted 10,000 rows
Chunk 27: inserted 10,000 rows
Chunk 28: inserted 10,000 rows
Chunk 29: inserted 10,000 rows
Chunk 30: inserted 10,000 rows
Chunk 31: inserted 10,000 rows
Chunk 32: inserted 10,000 rows
Chunk 33: inserte

In [31]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("SELECT COUNT(*) FROM bronze.us_accidents")
    )
    db_count = result.scalar()

print(f"Rows in bronze.us_accidents: {db_count:,}")

Rows in bronze.us_accidents: 7,718,394
